This notebook is looking at how accurate the decile scores are at predicting recidivism by using at logistic regression algorithm.

In [2]:
import pandas as pd
import numpy as np


In [70]:
compas = pd.read_csv("compas-scores-two-years.csv")

In [71]:
##data cleaning
colremove = ['id', 'name', 'first', 'last', 'compas_screening_date','dob','age_cat',
             'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas','r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out','vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment','screening_date',
       'v_type_of_assessment','v_screening_date', 'in_custody', 'out_custody','start', 'end', 'event',
       'c_charge_degree', 'c_charge_desc', "priors_count.1", "violent_recid"]
compas2 = compas.drop(colremove, axis =1)

In [72]:

pd.set_option('display.max_columns', None)


In [73]:
compas2.head()

,sex,age,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,is_recid,is_violent_recid,decile_score.1,score_text,v_decile_score,v_score_text,two_year_recid
0,Male,69,Other,0,1,0,0,0,0,0,1,Low,1,Low,0
1,Male,34,African-American,0,3,0,0,0,1,1,3,Low,1,Low,1
2,Male,24,African-American,0,4,0,1,4,1,0,4,Low,3,Low,1
3,Male,23,African-American,0,8,1,0,1,0,0,8,High,6,Medium,0
4,Male,43,Other,0,1,0,0,2,0,0,1,Low,1,Low,0


In [74]:
score_text = compas2[["score_text", "v_score_text"]]

In [75]:
compas2 = compas2.drop(["score_text", "v_score_text"], axis = 1)


In [76]:
compasml = pd.get_dummies(compas2)
compasml.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,is_recid,is_violent_recid,decile_score.1,v_decile_score,two_year_recid,sex_Female,sex_Male,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other
0,69,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1
1,34,0,3,0,0,0,1,1,3,1,1,0,1,1,0,0,0,0,0
2,24,0,4,0,1,4,1,0,4,3,1,0,1,1,0,0,0,0,0
3,23,0,8,1,0,1,0,0,8,6,0,0,1,1,0,0,0,0,0
4,43,0,1,0,0,2,0,0,1,1,0,0,1,0,0,0,0,0,1


In [77]:
numerical_data = compasml.select_dtypes(np.number)
numerical_data = numerical_data.drop(["is_recid", "is_violent_recid","decile_score.1"], axis = 1)

In [78]:
numerical_data.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,v_decile_score,two_year_recid,sex_Female,sex_Male,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other
0,69,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1
1,34,0,3,0,0,0,1,1,0,1,1,0,0,0,0,0
2,24,0,4,0,1,4,3,1,0,1,1,0,0,0,0,0
3,23,0,8,1,0,1,6,0,0,1,1,0,0,0,0,0
4,43,0,1,0,0,2,1,0,0,1,0,0,0,0,0,1


In [79]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [80]:
##making/running model
target_variable = "two_year_recid"
independent_variables = numerical_data.drop(columns="two_year_recid").columns

X = numerical_data[independent_variables]  # numpy array
y = numerical_data[target_variable]                 # numpy array
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [81]:
model = LogisticRegression()
model.fit(X=X_train, y=y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [82]:
predictions = model.predict(X_test)
true_classes = y_test
predictions_probabilities = model.predict_proba(X_test)

In [83]:
from sklearn import metrics

In [84]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_classes, predictions)

array([[624, 173],
       [256, 390]])

In [85]:
##accuracy
from sklearn import metrics
metrics.accuracy_score(true_classes, predictions)

0.7027027027027027

In [86]:
probs_df = pd.DataFrame(predictions_probabilities)
probs_df = round(probs_df, 2)
probs_df.head()

,0,1
0,0.65,0.35
1,0.43,0.57
2,0.31,0.69
3,0.75,0.25
4,0.72,0.28


In [89]:
X = X_test.reset_index().copy()
X["target"] = y_test.tolist()
X["prediction"] = predictions

X = X.rename(columns={'race_African-American': 'race_African_American'})

In [90]:
X.head(20)

,index,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,v_decile_score,sex_Female,sex_Male,race_African_American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other,target,prediction
0,1225,28,0,4,0,0,1,4,1,0,0,0,1,0,0,0,0,0
1,3269,33,0,8,0,0,2,9,0,1,1,0,0,0,0,0,1,1
2,4747,25,0,8,0,0,5,6,0,1,1,0,0,0,0,0,1,1
3,3324,38,0,4,0,0,0,1,1,0,1,0,0,0,0,0,0,0
4,1119,37,0,2,0,0,0,1,0,1,0,0,1,0,0,0,0,0
5,2273,37,0,2,0,0,1,3,0,1,0,0,1,0,0,0,0,0
6,6747,23,0,3,0,0,1,4,0,1,0,0,1,0,0,0,0,0
7,1141,25,0,3,0,0,1,3,0,1,0,0,1,0,0,0,1,0
8,6294,26,0,6,0,0,3,4,1,0,0,0,1,0,0,0,1,0
9,535,25,0,3,0,0,0,4,0,1,0,0,1,0,0,0,0,0


In [92]:
af = X[X.race_African_American == 1]
(af.target == af.prediction).mean()

0.720626631853786

In [93]:
wh = X[X.race_Caucasian == 1]
(wh.target == wh.prediction).mean()

0.6872246696035242

In [34]:
##redoing it with only decile scores
scores = compasml[["decile_score", "v_decile_score"]]
var = compasml["two_year_recid"]

In [35]:
X = scores  # numpy array
y = var                # numpy array
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [36]:
model.fit(X=X_train, y=y_train)
predictions2 = model.predict(X_test)
true_classes2 = y_test
predictions_probabilities2 = model.predict_proba(X_test)

In [37]:
confusion_matrix(true_classes2, predictions2)

array([[624, 191],
       [288, 340]])

In [38]:
metrics.accuracy_score(true_classes2, predictions2)

0.668052668052668

In [41]:
def logregress(data):
    compasml = pd.get_dummies(data)
    numerical_data = compasml.select_dtypes(np.number)
    numerical_data = numerical_data.drop(["is_recid", "is_violent_recid","decile_score.1"], axis = 1)
    target_variable = "two_year_recid"
    independent_variables = numerical_data.drop(columns="two_year_recid").columns
    X = numerical_data[independent_variables]  # numpy array
    y = numerical_data[target_variable]
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
    model.fit(X=X_train, y=y_train)
    predictions = model.predict(X_test)
    true_classes = y_test
    predictions_probabilities = model.predict_proba(X_test)
    return (confusion_matrix(true_classes, predictions),metrics.accuracy_score(true_classes, predictions))

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [59]:
###checking some things with race (didn't pan out)
afam = compas2[compas2.race == "African-American"]
cauc = compas2[compas2.race == "Caucasian"]
hisp = compas2[compas2.race == "Hispanic"]

In [60]:
logregress(afam)

(array([[263,  99],
        [135, 243]]), 0.6837837837837838)

In [61]:
logregress(cauc)

(array([[269,  37],
        [105,  80]]), 0.7107942973523421)

In [62]:
logregress(hisp)

(array([[73,  8],
        [35, 12]]), 0.6640625)

In [65]:
X[X["race_African-American"] == 1]

KeyError: 'race_African-American'